In [164]:
class PolynomialTerm:
    def __hash__(self):
        return hash(str(self))
    
    def __str__(self):
        return str(self.value)
    
    def is_number(self):
        return False
        
    def is_symbol(self):
        return False
    
    def could_extract_minus_sign(self):
        return False
    
    def __repr__(self):
        return str(self)
    
    def __hash__(self):
        return hash(str(self))  
    
    def __eq__(self, other):
        if not isinstance(other, PolynomialTerm):
            return False
        return str(self) == str(other)  

class Variable(PolynomialTerm):
    def __init__(self, value):
        self.value = value
        
    def is_symbol(self):
        return True
    
    def __pow__(self, exponent):
        if isinstance(exponent, Constant):
            return Power(self, exponent)
        elif isinstance(exponent, (int, float)):
            return Power(self, Constant(exponent))
        
    def __hash__(self):
        return hash(self.value)
    
    def __eq__(self, other):
        if not isinstance(other, Variable):
            return False
        return self.value == other.value
       


class Constant(PolynomialTerm):
    def __init__(self, value):
        self.value = float(value)
        
    def is_number(self):
        return True
        
    @property
    def is_integer(self):
        return self.value == int(self.value)
        
    def __gt__(self, second):
        if isinstance(second, Constant):
            return self.value > second.value
        return self.value > second
        
    def __eq__(self, second):
        if isinstance(second, Constant):
            return self.value == second.value
        return self.value == second
    
    def __hash__(self):
        return hash(self.value)
    
    def __mod__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value % other.value)
        return Constant(self.value % other)

    def __rmod__(self, other):
        return Constant(other % self.value)
    
    def __sub__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value - other.value)
        return Constant(self.value - other)

    def __rsub__(self, other):
        return Constant(other - self.value)
    
    def __pow__(self, exponent):
        if isinstance(exponent, Constant):
            return Constant(self.value ** exponent.value)
        return Constant(self.value ** exponent)
    
    def __floordiv__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value // other.value)
        return Constant(self.value // other)

    def __rfloordiv__(self, other):
        return Constant(other // self.value)


class BinaryOperation(PolynomialTerm):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    @property
    def args(self):
        return [self.left, self.right]


class Addition(BinaryOperation):
    def __str__(self):
        return f"{self.left} + {self.right}"
    

class Subtraction(BinaryOperation):
    def __str__(self):
        return f"{self.left} - {self.right}"
    

class Multiplication(BinaryOperation):
    def __str__(self):
        # Adds parentheses around additions and subtractions
        left_str = str(self.left)
        right_str = str(self.right)

        if isinstance(self.left, (Addition, Subtraction)):
            left_str = f"({left_str})"
            
        if isinstance(self.right, (Addition, Subtraction)):
            right_str = f"({right_str})"
            
        return f"{left_str} * {right_str}"
    

class Power(BinaryOperation):
    def __str__(self):
        # Adds parentheses around the base for clarity
        left_str = str(self.left)
        if isinstance(self.left, (Addition, Subtraction, Multiplication)):
            left_str = f"({left_str})"
            
        return f"{left_str}^{self.right}"
    

class UnaryMinus(PolynomialTerm):
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        term_str = str(self.value)
        if isinstance(self.value, (Addition, Subtraction, Multiplication)):
            term_str = f"({term_str})"
        return f"-{term_str}"
    
    def is_number(self):
        return self.value.is_number() if hasattr(self.value, 'is_number') else False
    
    def could_extract_minus_sign(self):
        return True
    
    @property
    def args(self):
        return [self.value]


class PolynomialParser:
    def __init__(self, expr_string):
        self.expr_tree = None
        self.parse(expr_string)

    
    def parse(self, expr):
        self.expr = expr.replace(' ', '') 
        self.pos = 0
        self.expr_tree = self.parse_expression()
        return self
    
    def current_char(self):
        if self.pos >= len(self.expr):
            return None
        return self.expr[self.pos]
    
    def next_char(self):
        if self.pos >= len(self.expr):
            return None
        return self.expr[self.pos + 1] if self.pos + 1 < len(self.expr) else None

    
    def parse_expression(self):
        if self.current_char() == '-' and self.next_char() and not self.next_char().isdigit():
            self.pos += 1
            left = Multiplication(Constant(-1.0), self.parse_term())
        else:
            left = self.parse_term()
        
        while self.current_char() in ('+', '-'):
            op = self.current_char()
            self.pos += 1
            right = self.parse_term()
            
            if op == '+':
                left = Addition(left, right)
            else:
                left = Subtraction(left, right)
        
        return left
    
    def parse_term(self):
        left = self.parse_factor()
        
        while self.current_char() == '*':
            self.pos += 1
            right = self.parse_factor()
            left = Multiplication(left, right)
                
        return left
    
    def parse_factor(self):
        char = self.current_char()
        
        # End of expression check
        if char is None:
            return None
        
        #  unary minus
        if char == '-' and self.next_char() and not self.next_char().isdigit():
            self.pos += 1
            return UnaryMinus(self.parse_factor())
        
        # Brackets
        if char == '(':
            self.pos += 1
            expr = self.parse_expression()
            self.pos += 1  # Skip closing bracket
            
            if self.current_char() == '^':
                self.pos += 1
                exponent = self.parse_factor()
                return Power(expr, exponent)
                
            return expr
        
        #  numbers
        if char.isdigit() or (char == '-' and self.next_char() and self.next_char().isdigit()):
            return self.parse_number()
        
        #  variables
        if char.isalpha():
            var = self.parse_variable()
            
            # Check for power after variable
            if self.current_char() == '^':
                self.pos += 1
                exponent = self.parse_factor()
                return Power(var, exponent)

            return var
            
    
    def parse_number(self):
        start_pos = self.pos
        char = self.current_char()
        
        # Handle negative sign only at the beginning
        if char == '-':
            self.pos += 1
            char = self.current_char()
        
        # Parse digits and decimal point
        while char is not None and (char.isdigit() or char == '.'):
            self.pos += 1
            char = self.current_char()
        
        # Create constant
        value = float(self.expr[start_pos:self.pos])
        return Constant(value)
    
    def parse_variable(self):
        start_pos = self.pos
        char = self.current_char()
        
        while char is not None and char.isalnum():
            self.pos += 1
            char = self.current_char()
            
        variable = self.expr[start_pos:self.pos]
        return Variable(variable)
    

    def count(self, node=None):

        if node is None:
            node = self.expr_tree
        
        return self.count_terms(node)
    
    def count_terms(self, node):

        if type(node) in (Addition, Subtraction):
            left_count = self.count_terms(node.left)
            right_count = self.count_terms(node.right)
            return left_count + right_count
        else:
            return 1
        
    def find_variables(self, node = None, variables = None):

        if variables is None:
            variables = set()

        if node is None:
            node = self.expr_tree

        if isinstance(node, Variable):
            variables.add(node.value)

        elif isinstance(node, (Addition, Subtraction, Multiplication, Power)):
            self.find_variables(node.left, variables)
            self.find_variables(node.right, variables)
        elif isinstance(node, UnaryMinus):
            self.find_variables(node.value, variables)
    
        return variables
    

        
       


In [165]:
def extract_co_kernels(expression):
    kernels = []
    cokernels = []
    kernel_cokernel_pairs = []
    
    # Collects the terms from the expression
    terms_and_positions = group_terms(expression)
    terms = [term for term, pos in terms_and_positions]
    term_positions = {term: pos for term, pos in terms_and_positions}

    # print(term_positions)
    
    # For all terms find all possible factors
    all_factors = set()
    
    for term in terms:
        term_factors = extract_factors(term)
        all_factors.update(term_factors)

    # These are all the possible co-kernels
    all_factors = list(all_factors)

    print("All terms: ", terms)
    print("All factors: ", all_factors)
    
    # Keep track of unique pairs to avoid duplicates
    seen_pairs = set()
    
    # Go through all the factors and see if it can be factored out
    for potential_co_kernels in all_factors:
        kernel_terms = []
        factorisable_terms = []
        factored_positions = []
        
        for term in terms:
            cokernel = try_factorising(term, potential_co_kernels)
            if cokernel is not None:
                kernel_terms.append(cokernel)
                factorisable_terms.append(term)
                factored_positions.append(term_positions[term])

        # A valid kernel must be factorisable from at least 2 terms
        if len(kernel_terms) >= 2:
            # Sort kernel terms by position
            kernel_terms_ordered = list(zip(factored_positions, kernel_terms, factorisable_terms))
            kernel_terms_ordered.sort(key=lambda x: x[0]) 

            print(kernel_terms_ordered)

            # Orders the kernel terms in the order they appear in the expression
            # A seperate list stores the indices of the terms that are part of the kernel
            kernel_terms = [term for _, term, _ in kernel_terms_ordered]
            kernel_term_positions = [pos for pos, _, _ in kernel_terms_ordered]

            print(kernel_terms)
            print(kernel_term_positions)
           
            print(type(kernel_terms[0]))
            kernel_expr = kernel_terms[0]
            for i in range(1, len(kernel_terms)):
                if type(kernel_terms[i]) == UnaryMinus:
                    kernel_expr = Subtraction(kernel_expr, kernel_terms[i].value)
                else:
                    kernel_expr = Addition(kernel_expr, kernel_terms[i])
            
            print("Co kernel expression: ", kernel_expr)
            print("Co kernel positions: ", kernel_term_positions)
            # Create a normalized representation for duplicate detection
            kernel_str = str(potential_co_kernels)
            cokernel_str = str(kernel_expr)
            
            # Sort the factorisable terms to create a consistent signature
            term_signatures = sorted([str(term) for term in factorisable_terms])
            pair_signature = (kernel_str, cokernel_str, tuple(term_signatures))
            
            # Only add if we haven't seen this exact factorisation before
            if pair_signature not in seen_pairs:
                seen_pairs.add(pair_signature)
                kernel_cokernel_pairs.append({
                    'cokernel': potential_co_kernels,
                    'kernel': kernel_expr,
                    'kernel_pos': kernel_term_positions,
                    'original_terms': factorisable_terms,
                })
    
    return kernel_cokernel_pairs

def group_terms(expression, position = 0):
    # Goes through and groups terms seperated by subtraction and addition. These are the groups
    terms = []
    
    if type(expression) == Addition:
        left_terms = group_terms(expression.left, position)
        terms.extend(left_terms)

        right_position = position + len(left_terms)
        right_terms = group_terms(expression.right, right_position)

        terms.extend(right_terms)
    elif type(expression) == Subtraction:
        left_terms = group_terms(expression.left, position)
        terms.extend(left_terms)

        right_position = position + len(left_terms)
        right_terms = group_terms(expression.right, right_position)

        for term, pos in right_terms:
            terms.append((UnaryMinus(term), pos))
    else:
        terms.append((expression, position))
    
    return terms

def extract_factors(term):
    factors = set()
    
    if type(term) == Variable:
        factors.add(term)
    elif type(term) == Multiplication:
        # Add individual factors
        left_factors = extract_factors(term.left)
        right_factors = extract_factors(term.right)

        factors.update(left_factors)
        factors.update(right_factors)
        
        # Do not add factorisable term if it is a constant
        if not type(term.left) == Constant:
            factors.add(term.left)

        factors.add(term.right)
        
    elif type(term) == Power:
        # Add base, meaning to the power of one
        factors.add(term.left)

        # Add all other powers, starting from power of 2. 
        if type(term.right) == Constant and term.right.value >= 2:
            for i in range(2, int(term.right.value)):
                factors.add(Power(term.left, Constant(i)))
    elif type(term) == UnaryMinus:
        # Extract factors from the inner term
        inner_factors = extract_factors(term.value)
        factors.update(inner_factors)

    return factors

def try_factorising(term, co_kernel):
    if equal_terms(term, co_kernel):
        return Constant(1.0)
    
    if type(term) == Multiplication:
        # In a multiplication the co kernel can match either side of the multiplication
        if equal_terms(term.left, co_kernel):
            return term.right
        elif equal_terms(term.right, co_kernel):
            return term.left
        
        # Try to factor out left side
        left_kernel = try_factorising(term.left, co_kernel)
        if left_kernel is not None:
            if type(left_kernel) == Constant and left_kernel.value == 1.0:
                return term.right
            else:
                return Multiplication(left_kernel, term.right)
        
        # Try to factor out right side
        right_kernel = try_factorising(term.right, co_kernel)
        if right_kernel is not None:
            if type(right_kernel) == Constant and right_kernel.value == 1.0:
                return term.left
            else:
                return Multiplication(term.left, right_kernel)
    
    elif type(term) == Power and type(co_kernel) == Power:
        # If term and co kernel are powers, than the exponent of the co kernel needs to be smaller than the term
        if (equal_terms(term.left, co_kernel.left) and term.right.value > co_kernel.right.value):
            # Factorise exponent 
            new_power = term.right.value - co_kernel.right.value
            if new_power == 1:
                return term.left
            else:
                return Power(term.left, Constant(new_power))
    
    # In the case where the co kernel does not have a power and has exponent of 1
    elif type(term) == Power:
        # Check if the bases match
        if equal_terms(term.left, co_kernel):
            if type(term.right) == Constant and term.right.value > 1:
                new_power = term.right.value - 1
                if new_power == 1:
                    return term.left
                else:
                    return Power(term.left, Constant(new_power))
    
    elif type(term) == UnaryMinus:
        inner_kernel = try_factorising(term.value, co_kernel)
        if inner_kernel is not None:
            return UnaryMinus(inner_kernel)
    
    return None

def equal_terms(term1, term2):
    if type(term1) != type(term2):
        return False
    
    if type(term1) == Variable:
        return term1.value == term2.value
    
    elif type(term1) == Constant:
        return (abs(term1.value - term2.value) < 1e-10)
    
    elif type(term1) in (Addition, Subtraction, Multiplication, Power):
        left_equals = equal_terms(term1.left, term2.left)
        right_equals = equal_terms(term1.right, term2.right)
        return (left_equals and right_equals)
    
    elif type(term1) == UnaryMinus:
        return equal_terms(term1.value, term2.value)

# Test code
test_expression = "(4*x^6*y*z) - (5*x^4*y) - (3*x^2*z^2) + (9*x^2*z^2*y)"

print(f"Expression: {test_expression}")
print("=" * 50)

parser = PolynomialParser(test_expression)
kernel_pairs = extract_co_kernels(parser.expr_tree)

print("Expression has length: ", parser.count())

for i, kernel_pair in enumerate(kernel_pairs):
    print(f"Pair {i}:")
    print(f"  Kernel:    {kernel_pair['kernel']}")
    print(f"  Co-kernel: {kernel_pair['cokernel']}")
    print(f"  Kernel Positions: {kernel_pair['kernel_pos']}")
    print("-" * 40)

Expression: (4*x^6*y*z) - (5*x^4*y) - (3*x^2*z^2) + (9*x^2*z^2*y)
All terms:  [4.0 * x^6.0 * y * z, -(5.0 * x^4.0 * y), -(3.0 * x^2.0 * z^2.0), 9.0 * x^2.0 * z^2.0 * y]
All factors:  [4.0 * x^6.0 * y, x^3.0, x, z, z^2.0, x^2.0, 5.0 * x^4.0, x^6.0, 9.0 * x^2.0 * z^2.0, 9.0 * x^2.0, y, 4.0 * x^6.0, x^4.0, x^5.0, 3.0 * x^2.0]
[(0, 4.0 * x^3.0 * y * z, 4.0 * x^6.0 * y * z), (1, -(5.0 * x * y), -(5.0 * x^4.0 * y))]
[4.0 * x^3.0 * y * z, -(5.0 * x * y)]
[0, 1]
<class '__main__.Multiplication'>
Co kernel expression:  4.0 * x^3.0 * y * z - 5.0 * x * y
Co kernel positions:  [0, 1]
[(0, 4.0 * x^5.0 * y * z, 4.0 * x^6.0 * y * z), (1, -(5.0 * x^3.0 * y), -(5.0 * x^4.0 * y)), (2, -(3.0 * x * z^2.0), -(3.0 * x^2.0 * z^2.0)), (3, 9.0 * x * z^2.0 * y, 9.0 * x^2.0 * z^2.0 * y)]
[4.0 * x^5.0 * y * z, -(5.0 * x^3.0 * y), -(3.0 * x * z^2.0), 9.0 * x * z^2.0 * y]
[0, 1, 2, 3]
<class '__main__.Multiplication'>
Co kernel expression:  4.0 * x^5.0 * y * z - 5.0 * x^3.0 * y - 3.0 * x * z^2.0 + 9.0 * x * z^2.0 *

In [166]:
import pandas as pd

def generate_polynomial_matrix(expression):

    # Initialize data structures
    variables = []
    terms = []
    signs = []
    
    # Extract terms from the expression
    parser = PolynomialParser("")
    parser.expr_tree = expression

    # term_breakdown = parser.get_term_breakdown()
    # terms = term_breakdown['terms']

    # print(term_breakdown['terms'])
        
    print("Terms are: ", terms)

    grouped_terms = group_terms(parser.expr_tree)

    terms = [m[0] for m in grouped_terms]

    # Find all unique variables in the expression
    variables = list(parser.find_variables())
    # variables.sort()  # Sort for consistent ordering
    
    # Build the matrix data
    matrix_data = []
    for term in terms:
        term_row, term_sign = _analyze_term(term, variables)
        term_row['sign'] = term_sign
        matrix_data.append(term_row)
        signs.append(term_sign)
    
    # Create pandas DataFrame
    df = pd.DataFrame(matrix_data)
    # Reorder columns to have variables first, then sign
    column_order = variables + ['sign']
    df = df[column_order]
    
    return {
        'dataframe': df,
        'variables': variables,
        'terms': [str(term) for term in terms],
        'signs': signs,
        'num_terms': len(terms),
        'num_variables': len(variables)
    }

def find_all_variables(node):
    
    variables = set()
    
    if isinstance(node, Variable):
        variables.add(node.value)
    elif isinstance(node, (Addition, Subtraction, Multiplication, Power)):
        variables.update(find_all_variables(node.left))
        variables.update(find_all_variables(node.right))
    elif isinstance(node, UnaryMinus):
        variables.update(find_all_variables(node.value))


    return list(variables)

def _analyze_term(term, variables):
    """
    Analyze a single term to determine variable powers and sign.
    Returns (powers_dict, sign) where powers_dict has variable: power pairs.
    """
    powers = {var: 0 for var in variables}
    sign = 1
    
    # Handle negative terms
    if isinstance(term, UnaryMinus):
        sign = -1
        term = term.value
    
    # Extract variable powers from the term
    var_powers = _extract_variable_powers(term)
    
    # Update the powers dictionary
    for var_name, power in var_powers.items():
        if var_name in powers:
            powers[var_name] = power
    
    return powers, sign

def _extract_variable_powers(node):
    """
    Extract variable powers from a term node.
    Returns a dictionary {variable_name: power}.
    """
    var_powers = {}
    
    if isinstance(node, Variable):
        var_powers[node.value] = 1
    elif isinstance(node, Constant):
        # Constants don't contribute to variable powers
        pass
    elif isinstance(node, Multiplication):
        # Combine powers from both sides
        left_powers = _extract_variable_powers(node.left)
        right_powers = _extract_variable_powers(node.right)
        
        # Merge the dictionaries, adding powers for same variables
        for var, power in left_powers.items():
            var_powers[var] = var_powers.get(var, 0) + power
        for var, power in right_powers.items():
            var_powers[var] = var_powers.get(var, 0) + power
            
    elif isinstance(node, Power):
        # Get the base variable powers and multiply by exponent
        base_powers = _extract_variable_powers(node.left)
        
        # Extract exponent value
        if isinstance(node.right, Constant):
            exp_value = int(node.right.value)
            for var, power in base_powers.items():
                var_powers[var] = power * exp_value
        else:
            # If exponent is not a constant, treat as power 1 for simplicity
            for var, power in base_powers.items():
                var_powers[var] = power
    elif isinstance(node, UnaryMinus):
        # Extract from the inner term (sign handled elsewhere)
        var_powers = _extract_variable_powers(node.value)
    
    return var_powers

def print_matrix(matrix_data):
    """
    Print the DataFrame in a readable format.
    """
    print("\nPolynomial DataFrame Representation:")
    print("=" * 60)
    
    # Create a copy of the DataFrame for display
    display_df = matrix_data['dataframe'].copy()
    display_df.index = matrix_data['terms']
    
    # Format the sign column for display
    display_df['sign'] = display_df['sign'].apply(lambda x: '+' if x > 0 else '-')
    
    print(display_df)
    print(f"\nDataFrame dimensions: {matrix_data['num_terms']} terms × {matrix_data['num_variables']} variables")
    print(f"Variables: {', '.join(matrix_data['variables'])}")

def get_dataframe(matrix_data):
    """
    Return the matrix data as a pandas DataFrame with proper headers.
    """
    df = matrix_data['dataframe'].copy()
    df.index = matrix_data['terms']
    return df


# Test code
test_expressions = [
    "a*b + c*d - c^2*d",
    "x + y - z",
    "a*x^2 + b*x + c",
    "2*a*b + 3*a*c - a*d + e*f",
    "x^2*y + x*y^2 - x*y + y^2",
    "a + b*c + d*e*f - g^3"
]

for expr_str in test_expressions:
    print(f"\n{'='*80}")
    print(f"Expression: {expr_str}")
    print('='*80)
    
    # Parse the expression
    parser = PolynomialParser(expr_str)
    
    # Generate matrix
    matrix_data = generate_polynomial_matrix(parser.expr_tree)
    
    # Print the matrix
    print_matrix(matrix_data)
    
    # Show DataFrame
    print("\nDataFrame details:")
    df = get_dataframe(matrix_data)
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("Data:")
    print(df)


Expression: a*b + c*d - c^2*d
Terms are:  []

Polynomial DataFrame Representation:
              b  d  c  a sign
a * b         1  0  0  1    +
c * d         0  1  1  0    +
-(c^2.0 * d)  0  1  2  0    -

DataFrame dimensions: 3 terms × 4 variables
Variables: b, d, c, a

DataFrame details:
Shape: (3, 5)
Columns: ['b', 'd', 'c', 'a', 'sign']
Data:
              b  d  c  a  sign
a * b         1  0  0  1     1
c * d         0  1  1  0     1
-(c^2.0 * d)  0  1  2  0    -1

Expression: x + y - z
Terms are:  []

Polynomial DataFrame Representation:
    x  y  z sign
x   1  0  0    +
y   0  1  0    +
-z  0  0  1    -

DataFrame dimensions: 3 terms × 3 variables
Variables: x, y, z

DataFrame details:
Shape: (3, 4)
Columns: ['x', 'y', 'z', 'sign']
Data:
    x  y  z  sign
x   1  0  0     1
y   0  1  0     1
-z  0  0  1    -1

Expression: a*x^2 + b*x + c
Terms are:  []

Polynomial DataFrame Representation:
           b  x  c  a sign
a * x^2.0  0  2  0  1    +
b * x      1  1  0  0    +
c          